## Imports

In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import torch
import random

import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt

from tqdm import tqdm
from SleepClassBbox import Classifier
from dateutil.rrule import *
from torch.utils.data import Dataset, DataLoader

## Define dataset

In [2]:
class ClassDataset(Dataset):
    def __init__(
        self,
        ids,
        feat_indxs
    ):  
        self.feat_indxs = feat_indxs
        self.dir = 'C:/Users/Dawson/Kaggle Projects/Detect Sleep States/sleep_institute_data/chunked_data/'
        self.series_files = []
        self.event_files = []
        for id in ids:
            for f_name in os.listdir(self.dir):
                if f_name.startswith(id) and f_name.endswith('.npy'):
                    self.series_files.append(f_name)
                    self.event_files.append(f_name)
        
        for f_name in self.series_files[:]:
            if 'events' in f_name:
                self.series_files.remove(f_name)

        for f_name in self.event_files[:]:
            if 'events' not in f_name:
                self.event_files.remove(f_name)
    
    def __len__(self):
        return len(self.series_files)

    def __getitem__(self, index):
        series_data = np.load(self.dir + self.series_files[index])
        event_data = np.load(self.dir + self.event_files[index])

        # Feature indices:
        # 0: anglez 1: enmo 2: ang_min 3:ang_max 4: ang_std 5: ang_mean
        # 6: enmo_min 7: enmo_max 8: end_std 9: enmo_mean 10: class labels
        x = torch.Tensor(series_data[self.feat_indxs, :]).to(torch.float32)
        y_class = torch.Tensor(series_data[-1, :]).to(torch.float32)

        y_bbox = torch.Tensor([event_data[0], event_data[1]])/17280

        if torch.isnan(y_bbox).any():
            y_bbox = torch.nan_to_num(y_bbox)

        return x, y_class, y_bbox

## Training functions

In [3]:
def train(model, device, dataloader, cls_criterion, bbox_criterion, optimizer, scheduler, epoch, scaler):
    model.train()
    data_len = len(dataloader.dataset)
    running_cls_loss = 0
    running_bbox_loss = 0
    running_accuracy = 0
    num_batches = 0
    for batch_idx, data in tqdm(enumerate(dataloader)):
        accel_data, cls_labels, bbox_labels = data

        # Scale and clip acceleration data
        accel_data[:, 0, :] = accel_data[:, 0, :]/100
        accel_data[:, 1, :] = torch.clamp(accel_data[:, 1, :], max=1)

        # Data augmentation
        # Flip data along time axis
        flip = torch.randn((1))
        if flip.item() > 0.5:
            accel_data = torch.flip(accel_data, (2,))
            cls_labels = torch.flip(cls_labels, (1,))
            bbox_labels = torch.flip(bbox_labels, (1,))

        # Roll data along time axis
        shift_size = torch.randint(0, 17280, size=(1,))
        accel_data = torch.roll(accel_data, (shift_size[0],), 2)
        cls_labels = torch.roll(cls_labels, (shift_size[0],), 1)
        bbox_labels = torch.roll(bbox_labels, (shift_size[0],), 1)

        accel_data, cls_labels, bbox_labels = accel_data.to(device), cls_labels.to(device), bbox_labels.to(device)
 
        # one-hot encode labels
        cls_labels = F.one_hot(cls_labels.to(torch.int64), num_classes=3).transpose(1, 2).to(torch.float32) # (batch, # classes, time)

        optimizer.zero_grad(set_to_none=True)
        torch.autograd.set_detect_anomaly(True) 
        with torch.autocast(device_type=str(device), dtype=torch.bfloat16):
            cls_output, bbox_output = model(accel_data) # (batch, # classes, time), (batch, 2, time)
            cls_output = F.softmax(cls_output, dim=1)
            bbox_output = F.sigmoid(bbox_output)

            cls_loss = cls_criterion(cls_output, cls_labels)
            bbox_loss = bbox_criterion(bbox_output, bbox_labels)

            loss = bbox_loss 
            
            # loss.backward()
            scaler.scale(loss).backward()

        running_cls_loss += cls_loss.item()
        running_bbox_loss += bbox_loss.item()

        preds = torch.argmax(cls_output, dim=1)
        arg_labels = torch.argmax(cls_labels, dim=1)
        running_accuracy += torch.sum(preds == arg_labels)/torch.numel(preds)
        num_batches += 1

        scaler.step(optimizer)
        scaler.update()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5) # Clip gradients
        scheduler.step()

        if (batch_idx+1) % 200 == 0 or batch_idx == (data_len//accel_data.shape[0])-1:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]  Class Loss: {:.6f}  BBox Loss: {:.6f}  Accuracy: {:.6f}'.format(
                epoch, batch_idx * len(accel_data), data_len,
                100 * batch_idx / len(dataloader), running_cls_loss/num_batches,
                running_bbox_loss/num_batches,  running_accuracy/num_batches
            ))
            running_cls_loss = 0
            running_bbox_loss = 0
            running_accuracy = 0
            num_batches = 0
    
    return model, optimizer

def validate(model, device, dataloader, cls_criterion, crt_criterion):
    model.eval()
    running_cls_loss = 0
    running_bbox_loss = 0
    running_accuracy = 0
    for batch_idx, data in tqdm(enumerate(dataloader)):
        accel_data, cls_labels, bbox_labels = data

        # Scale and clip acceleration data
        accel_data[:, 0, :] = accel_data[:, 0, :]/100
        accel_data[:, 1, :] = torch.clamp(accel_data[:, 1, :], max=1)

        accel_data, cls_labels, bbox_labels = accel_data.to(device), cls_labels.to(device), bbox_labels.to(device)

        # one-hot encode labels
        cls_labels = F.one_hot(cls_labels.to(torch.int64), num_classes=3).transpose(1, 2).to(torch.float32) # (batch, # classes, time)

        with torch.no_grad():
            with torch.autocast(device_type=str(device), dtype=torch.bfloat16):
                cls_output, bbox_output = model(accel_data) # (batch, # classes, time)
                cls_output = F.softmax(cls_output, dim=1)
                bbox_output = F.sigmoid(bbox_output)

                cls_loss = cls_criterion(cls_output, cls_labels)
                crt_loss = crt_criterion(bbox_output, bbox_labels)

        running_cls_loss += cls_loss.item()
        running_bbox_loss += crt_loss.item()
        preds = torch.argmax(cls_output, dim=1)
        arg_labels = torch.argmax(cls_labels, dim=1)
        running_accuracy += torch.sum(preds == arg_labels)/torch.numel(preds)

    print('Validation - Class Loss: {:.6f}  Crit Loss: {:.6f}  Accuracy: {:.6f}'.format(running_cls_loss/(batch_idx+1), 
                                                                                        running_bbox_loss/(batch_idx+1),
                                                                                        running_accuracy/(batch_idx+1)
                                                                                        ))
    
    torch.cuda.empty_cache()

def evaluate(model, device, dataloader, mode='hard'):
    model.eval()
    predictions = []
    crit_preds = []
    sleep_probs = []
    awake_probs = []
    for batch_idx, data in tqdm(enumerate(dataloader)):
        accel_data, cls_labels, crt_labels = data

        # Scale and clip acceleration data
        accel_data[:, 0, :] = accel_data[:, 0, :]/100
        accel_data[:, 1, :] = torch.clamp(accel_data[:, 1, :], max=1)

        accel_data = accel_data.to(device)
        
        with torch.no_grad():
            with torch.autocast(device_type=str(device), dtype=torch.bfloat16):
                cls_output, bbox_output = model(accel_data) # (batch, # classes, time)
                cls_output = F.softmax(cls_output, dim=1)
                bbox_output = F.sigmoid(bbox_output)
                
        if mode == 'hard':
            predictions.append(torch.squeeze(torch.argmax(cls_output, dim=1)))
            crit_preds.append(bbox_output)
            sleep_probs.append(torch.squeeze(cls_output[:, 1, :]))
            awake_probs.append(torch.squeeze(cls_output[:, 0, :]))
        else:
            predictions.append(cls_output)
    
    return predictions, crit_preds, sleep_probs, awake_probs


In [4]:
def train_and_validation(hparams):
    
    print('Initializing datasets...')
    # Define datasets
    unique_ids = list(set([filename.split('_')[0] for filename in os.listdir('C:/Users/Dawson/Kaggle Projects/Detect Sleep States/sleep_institute_data/chunked_data')]))
    rand_indxs = random.sample(list(range(len(unique_ids))), len(unique_ids))
    rand_ids = [unique_ids[indx] for indx in rand_indxs]
    train_ids = rand_ids[0:len(rand_indxs) - 50]
    valid_ids = rand_ids[len(rand_indxs) - 50:]

    train_ds = ClassDataset(train_ids, [0,1])
    valid_ds = ClassDataset(valid_ids, [0,1])
    
    print('Computing label weights...')
    # Get label weights
    label_weights = np.zeros((3))
    num_examples = 0
    for day in train_ds:
        day_label = day[1]
        label_weights[0] += torch.where(day_label == 0)[0].shape[0]/day_label.shape[0]
        label_weights[1] += torch.where(day_label == 1)[0].shape[0]/day_label.shape[0]
        label_weights[2] += torch.where(day_label == 2)[0].shape[0]/day_label.shape[0]
        num_examples += 1
    label_weights /= num_examples
    label_weights = torch.Tensor(label_weights)
    print(1/label_weights)

    use_cuda = torch.cuda.is_available()
    device = torch.device('cuda' if use_cuda else 'cpu')
    print('Training device: ' + str(device))

    kwargs = {'num_workers': 0, 'pin_memory': True} if use_cuda else {}
    train_loader = DataLoader(dataset=train_ds,
                             batch_size=hparams['batch size'],
                             shuffle=True,
                             **kwargs)

    valid_loader = DataLoader(dataset=valid_ds,
                             batch_size=hparams['batch size'],
                             shuffle=False,
                              **kwargs)
    
    model = Classifier(2, feat_sizes=hparams['feat sizes'], depths=hparams['depths']).to(device)

    print('Number of model parameters:', sum([param.nelement() for param in model.parameters()]))

    optimizer = optim.AdamW(model.parameters(), hparams['lr'])
    cls_criterion = nn.CrossEntropyLoss(weight=1/label_weights).to(device)
    crt_criterion = nn.L1Loss().to(device)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=hparams['lr'],
                                              steps_per_epoch=int(len(train_loader)),
                                              epochs=hparams['epochs'],
                                              anneal_strategy='cos',
                                              pct_start=hparams['pct start'],
                                              div_factor=hparams['start factor'],
                                              final_div_factor=hparams['final factor'])
    
    scaler = torch.cuda.amp.GradScaler()
    for epoch in range(hparams['epochs']):
        trained_model, optimizer = train(model, device, train_loader, cls_criterion, crt_criterion, optimizer, scheduler, epoch, scaler)
        validate(model, device, valid_loader, cls_criterion, crt_criterion)
    
    return trained_model.state_dict(), optimizer.state_dict(), hparams, valid_ds


## Train ensemble of models

In [5]:
def train_ensemble(num_models, hparams):
    """
    Train num_models models and save each model to respective files. hparams can be a list of
    dictionaries which can be indexed with i.
    """

    for i in range(num_models):
        print('Training model ' + str(i+1))
        model_state_dict, optim_state_dict, model_hparams, valid_ds = train_and_validation(hparams)

        torch.save({
            'model_state_dict': model_state_dict,
            'optimizer_state_dict': optim_state_dict,
            'model_hparams': model_hparams,
        }, './trained_models/model_' + str(i+1) + '.tar')

    return valid_ds

In [6]:
depths_list = [(8,), (9,), (10,), (11,), (12,), (13,)]

hparams = {
    'epochs': 10,
    'batch size': 16,
    'lr': 1e-1,
    'pct start': 0.1,
    'start factor': 1e2,
    'final factor': 1e3,
    'feat sizes': (64,),
    'depths': (14,)
}
torch.cuda.empty_cache()
eval_ds = train_ensemble(1, hparams)

Training model 1
Initializing datasets...
Computing label weights...
tensor([2.1131, 4.2016, 3.4632])
Training device: cuda
Number of model parameters: 1381401


200it [00:52,  3.90it/s]

Train Epoch: 0 [3184/5734 (55%)]  Class Loss: 3.297898  BBox Loss: 0.193414  Accuracy: 0.258861


358it [01:33,  3.91it/s]

Train Epoch: 0 [5712/5734 (99%)]  Class Loss: 3.293508  BBox Loss: 0.197321  Accuracy: 0.229400


...

Train Epoch: 9 [3184/5734 (55%)]  Class Loss: 3.291539  BBox Loss: 0.154999  Accuracy: 0.306487


359it [01:31,  4.21it/s]

Train Epoch: 9 [5712/5734 (99%)]  Class Loss: 3.300860  BBox Loss: 0.157350  Accuracy: 0.308131


359it [01:31,  3.91it/s]
84it [00:04, 17.75it/s]

Validation - Class Loss: 3.233306  Crit Loss: 0.167196  Accuracy: 0.376163


## Ensemble predictions

In [ ]:
def ensemble_predict(dataloader, mode='hard'):
    """
    Loads models and evaluates the given dataset. Predictions are then voted on with either 'hard' or
    'soft' voting depending on mode selection
    """
    # Various initilizations
    use_cuda = torch.cuda.is_available()
    device = torch.device('cuda' if use_cuda else 'cpu')
    print('Predicting with ' + str(device))

    model_files = os.listdir('./trained_models')

    # Iterate over models and make predictions
    model_num = 0
    if mode == 'hard':
        pred_list = []
        crit_points = []
        sleep_probs = []
        awake_probs = []
        for file in model_files:
            print('Evaluating model ' + str(model_num + 1))
            checkpoint = torch.load('./trained_models/' + file)
            hparams = checkpoint['model_hparams']
            model = Classifier(10, feat_sizes=hparams['feat sizes'], depths=hparams['depths']).to(device)
            model.load_state_dict(checkpoint['model_state_dict'])

            predictions, crt_out, slp_probs, awk_probs = evaluate(model, device, dataloader, mode=mode)
            pred_list.append(predictions)
            crit_points.append(crt_out)
            sleep_probs.append(slp_probs)
            awake_probs.append(awk_probs)
            model_num += 1
            
        return pred_list, crit_points, sleep_probs, awake_probs
    if mode == 'soft':
        voted_preds = []
        crt_points = []
        for example, data in tqdm(enumerate(dataloader)):
            pred_sums = torch.zeros((1, 3, 17280)).to(device)
            crt_sums = torch.zeros((1, 2, 17280)).to(device)
            model_num = 0
            input, label = data
            input = input /100
            for file in model_files:
                checkpoint = torch.load('./trained_models/' + file)
                hparams = checkpoint['model_hparams']
                model = Classifier(10, feat_sizes=hparams['feat sizes'], depths=hparams['depths']).to(device)
                model.load_state_dict(checkpoint['model_state_dict'])
                
                model.eval()
                with torch.no_grad():
                    with torch.autocast(device_type=str(device), dtype=torch.float16):
                        out, crt_out = model(input.to(device))
                        out = F.softmax(out, dim=1)
                        pred_sums += out
                        crt_sums += crt_out
                model_num += 1
            
            voted_preds.append(torch.argmax(pred_sums, dim=1))
            crt_points.append(crt_sums)

        return voted_preds, crt_points

def ensemble_voting(predictions, mode='hard'):
    """
    Takes a nested list of model predictions and computes voting results depending on
    mode selection.
    """

    # Stack predictions from each model
    stacked_preds = []
    for i in range(len(predictions)):
        stacked_preds.append(torch.stack(predictions[i], dim=0))

    # Stack predictions from all models
    stacked_preds = torch.stack(stacked_preds, dim=0).to('cpu') # (model num, num days, num steps) when mode = hard
                                                      # (model num, num classes, num days, num steps) when mode = soft

    if mode == 'hard':
        num_votes = torch.zeros((3, stacked_preds.shape[1], stacked_preds.shape[2]))

        # Awake votes
        indxs = torch.where(stacked_preds == 0)
        num_votes[0, indxs[1], indxs[2]] += 1
        # Asleep votes
        indxs = torch.where(stacked_preds == 1)
        num_votes[1, indxs[1], indxs[2]] += 1
        # Not wearing votes
        indxs = torch.where(stacked_preds == 2)
        num_votes[2, indxs[1], indxs[2]] += 1

        voted_predictions = torch.argmax(num_votes, dim=0) # (num_days, num steps)
    
    if mode == 'soft':
        summed_preds = torch.sum(stacked_preds, dim=0) # (num classes, num days, num steps)
        voted_predictions = torch.argmax(summed_preds, dim=0) # (num days, num steps)
    
    return voted_predictions

def crit_point_average(crit_points):
    stacked_crits = []
    for i in range(len(crit_points)):
        stacked_crits.append(torch.stack(crit_points[i], dim=0))
    stacked_crits = torch.stack(stacked_crits, dim=0)
    avg_crits = torch.mean(stacked_crits, dim=0)
    return torch.squeeze(avg_crits)

In [ ]:
# # Define datasets
# unique_ids = list(set([filename.split('_')[0] for filename in os.listdir('C:/Users/Dawson/Kaggle Projects/Detect Sleep States/sleep_institute_data/chunked_data')]))
# rand_indxs = random.sample(list(range(len(unique_ids))), len(unique_ids))
# rand_ids = [unique_ids[indx] for indx in rand_indxs]
# eval_ids = rand_ids[len(rand_indxs) - 25:]

# eval_ds = ClassDataset(eval_ids, [0,1])

eval_loader = DataLoader(dataset=eval_ds,
                        batch_size=1,
                        shuffle=False)

predictions, crit_points, sleep_probabilities, awake_probabilities = ensemble_predict(eval_loader, mode='hard')
vote_preds = ensemble_voting(predictions, mode='hard')
crit_points = crit_point_average(crit_points)

## Post-processing

In [ ]:
def get_longer_list(lists):
    """
    Returns the larger of two lists
    """
    lengths = [len(lst) for lst in lists]
    return lists[np.argmax(lengths)]

In [ ]:
def process_prediction(prediction):
    """
    Processes a single 24 hour period of predictions to output the step numbers of 'onset', 'wakeup', or nan
    within that day.
    """
    
    # Get indices where subject is predicted to be asleep and awake
    sleep_indxs = np.where(prediction == 1)[0]
    awake_indxs = np.where(prediction == 0)[0]

    # Return nans if awake for 24 hours
    if (len(sleep_indxs) == 0) | (len(awake_indxs) == 0):
        return np.nan, np.nan

    # Determine duration of predicted sleeping and waking windows
    sleep_windows = np.split(sleep_indxs, np.where(np.diff(sleep_indxs, prepend=sleep_indxs[0]-1) != 1)[0])
    sleep_duration = [array.shape[0] for array in sleep_windows]

    awake_windows = np.split(awake_indxs, np.where(np.diff(awake_indxs, prepend=awake_indxs[0]-1) != 1)[0])
    awake_duration = [array.shape[0] for array in awake_windows]

    # Ignore windows shorter than 30 minutes
    half_hour_length = 360  # Number of steps for half an hour assuming 5 seconds per step
    valid_indxs = [i for i in range(len(sleep_duration)) if sleep_duration[i] > half_hour_length]
    valid_sleep_windows = [sleep_windows[valid_indxs[i]] for i in range(len(valid_indxs))]
    valid_indxs = [i for i in range(len(awake_duration)) if awake_duration[i] > half_hour_length]
    valid_awake_windows = [awake_windows[valid_indxs[i]] for i in range(len(valid_indxs))]
    
    # If significant portion of predictions have nan label return nans and no valid sleep windows
    # nan_ratio = np.where(prediction == 2)[0].shape[0]/prediction.shape[0]
    # if (nan_ratio >= 0.5) & (len(valid_sleep_windows)==0):
    #     return np.nan, np.nan


    # Return nans if no valid sleep windows are found
    if len(valid_sleep_windows) == 0:
        return np.nan, np.nan

    # Check if any valid awake windows lie between valid sleep windows
    chosen_windows = []
    for i in range(len(valid_sleep_windows)-1):
        for j in range(len(valid_awake_windows)):
            # Condition for if valid awake window is between two valid sleep windows
            if (valid_awake_windows[j][0] > valid_sleep_windows[i][-1]) & \
               (valid_awake_windows[j][-1] < valid_sleep_windows[i+1][0]):
                # Chose larger of two valid sleep windows
                chosen_windows.append(get_longer_list([valid_sleep_windows[i], 
                                                       valid_sleep_windows[i+1]]))

    # Assign onset and wakeup steps if no valid waking windows are found between valid sleep windows
    if len(chosen_windows) == 0:
        onset_step = valid_sleep_windows[0][0]
        wakeup_step = valid_sleep_windows[-1][-1]
    else: # If valid wake windows found choose largest sleep window
        largest_window = get_longer_list(chosen_windows)
        onset_step = largest_window[0]
        wakeup_step = largest_window[-1]

    return onset_step, wakeup_step

In [ ]:
index = 7
def plot_predictions(index):
    print('Plotting example number ' + str(index))
    prediction = vote_preds[index].to('cpu').numpy()
    onset_crts = crit_points[index, 0, :].to(torch.float16).to('cpu').numpy()
    wakeup_crts = crit_points[index, 1, :].to(torch.float16).to('cpu').numpy()
    onset_step, wakeup_step = process_prediction(prediction)

    fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, ncols=1, sharex=True, squeeze=True, figsize=(16, 8))
    steps = np.arange(eval_ds[index][1].shape[0])
    labels = eval_ds[index][1].numpy()
    onset_crt_label = eval_ds[index][2][0, :].numpy()
    wakeup_crt_label = eval_ds[index][2][1, :].numpy()

    accuracy = np.sum(np.array([prediction == labels]))/prediction.shape[0]
    print('Example accuracy = {:.4f}'.format(accuracy))

    ax1.plot(steps, eval_ds[index][0][0, :]/100)

    sum_slp_prob = torch.zeros_like(sleep_probabilities[0][0]).to('cpu')
    sum_awk_prob = torch.zeros_like(sleep_probabilities[0][0]).to('cpu')
    for i in range(len(sleep_probabilities)):
        sum_slp_prob += sleep_probabilities[i][index].to('cpu')
        sum_awk_prob += awake_probabilities[i][index].to('cpu')
    avg_slp_prob = sum_slp_prob / len(sleep_probabilities)
    avg_awk_prob = sum_awk_prob / len(awake_probabilities)


    # ax1.plot(steps, sleep_probabilities[0][index].to('cpu'))
    # ax1.plot(steps, sleep_probabilities[1][index].to('cpu'))
    # ax1.plot(steps, sleep_probabilities[2][index].to('cpu'))
    # ax1.plot(steps, sleep_probabilities[3][index].to('cpu'))
    # ax1.plot(steps, sleep_probabilities[4][index].to('cpu'))
    ax1.plot(steps, avg_slp_prob)
    ax1.plot(steps, avg_awk_prob, '--')

    ax1.fill_between(steps, y1=1, y2=0, where=prediction==0, facecolor='green', alpha=.5)
    ax1.fill_between(steps, y1=1, y2=0, where=prediction==1, facecolor='red', alpha=.5)
    ax1.fill_between(steps, y1=1, y2=0, where=prediction==2, facecolor='blue', alpha=.5)
    ax1.fill_between(steps, y1=0, y2=-1, where=labels==0, facecolor='lime', alpha=.5)
    ax1.fill_between(steps, y1=0, y2=-1, where=labels==1, facecolor='orangered', alpha=.5)
    ax1.fill_between(steps, y1=0, y2=-1, where=labels==2, facecolor='blue', alpha=.5)
    ax1.vlines(onset_step, -1, 1, colors='r', label='onset')
    ax1.vlines(wakeup_step, -1, 1, colors='b', label='wakeup')
    ax1.set_ylabel('anglez freq')

    try:
        print('onset diff (min): ' + str(np.abs(onset_step - steps[np.where((labels==1))[-1]][0])*5/60))
        print('wakeup diff (min): ' + str(np.abs(wakeup_step - steps[np.where((labels==1))[-1]][-1])*5/60))
    except:
        print('No valid sleep window')

    ax2.plot(steps, eval_ds[index][0][1, :])

    # ax2.plot(steps, sleep_probabilities[0][index].to('cpu'))
    # ax2.plot(steps, sleep_probabilities[1][index].to('cpu'))
    # ax2.plot(steps, sleep_probabilities[2][index].to('cpu'))
    # ax2.plot(steps, sleep_probabilities[3][index].to('cpu'))
    # ax2.plot(steps, sleep_probabilities[4][index].to('cpu'))
    ax2.plot(steps, avg_slp_prob)
    ax2.plot(steps, avg_awk_prob, '--')

    ax2.fill_between(steps, y1=2, y2=0, where=prediction==0, facecolor='green', alpha=.5)
    ax2.fill_between(steps, y1=2, y2=0, where=prediction==1, facecolor='red', alpha=.5)
    ax2.fill_between(steps, y1=2, y2=0, where=prediction==2, facecolor='blue', alpha=.5)
    ax2.fill_between(steps, y1=3, y2=2, where=labels==0, facecolor='lime', alpha=.5)
    ax2.fill_between(steps, y1=3, y2=2, where=labels==1, facecolor='orangered', alpha=.5)
    ax2.fill_between(steps, y1=3, y2=2, where=labels==2, facecolor='blue', alpha=.5)
    ax2.vlines(onset_step, 0, 3, colors='r')
    ax2.vlines(wakeup_step, 0, 3, colors='b')
    ax2.set_xlabel('step')
    ax2.set_ylabel('enmo')

    ax3.plot(steps, onset_crts, c='orangered')
    ax3.plot(steps, wakeup_crts, c='lime')
    ax3.plot(steps, onset_crt_label)
    ax3.plot(steps, wakeup_crt_label)
    ax3.fill_between(steps, y1=0, y2=7, where=labels==1, facecolor='orangered', alpha=.5)
    ax3.fill_between(steps, y1=0, y2=7, where=labels==2, facecolor='blue', alpha=.5)


    onset_crts[onset_crts < 2] = 0
    wakeup_crts[wakeup_crts < 2] = 0
    onset_crts[onset_crts > 10] = 0
    wakeup_crts[wakeup_crts > 10] = 0
    onset_cent = np.sum(onset_crts*steps)/np.sum(onset_crts)
    wakeup_cent = np.sum(wakeup_crts*steps)/np.sum(wakeup_crts)

    try:
        print('onset cent diff (min): ' + str(np.abs(onset_cent - steps[np.where((labels==1))[-1]][0])*5/60))
        print('wakeup cent diff (min): ' + str(np.abs(wakeup_cent - steps[np.where((labels==1))[-1]][-1])*5/60))
    except:
        None

    ax3.vlines(onset_cent, 0, 8, colors='r', label='onset')
    ax3.vlines(wakeup_cent, 0, 8, colors='b', label='wakeup')
    ax3.set_ylim([0, 8])

    plt.show()
    plt.close()

In [ ]:
plot_predictions(index)
index += 1

## Evaluate whole series

In [ ]:
def evaluate_series(model, device, series, mode='hard'):
    model.eval()

    # Scale and clip acceleration data
    series[:, 0, :] = series[:, 0, :]/100
    series[:, 1, :] = torch.clamp(series[:, 1, :], max=1)

    series = series.to(device)
    
    with torch.autocast(device_type=str(device), dtype=torch.float16):
        output = model(series) # (batch, # classes, time)
        output = F.softmax(output, dim=1)

    if mode == 'hard':
        predictions = torch.squeeze(torch.argmax(output, dim=1))
    else:
        predictions = torch.squeeze(output)
    
    return [predictions]

def series_predict(series, mode='hard'):
    """
    Loads models and evaluates an entire series. Predictions are then voted on with either 'hard' or
    'soft' voting depending on mode selection
    """
    # Various initilizations
    use_cuda = torch.cuda.is_available()
    device = torch.device('cuda' if use_cuda else 'cpu')
    print('Predicting with ' + str(device))

    model_files = os.listdir('./trained_models')

    # Iterate over models and make predictions
    pred_list = []
    model_num = 0
    for file in model_files:
        checkpoint = torch.load('./trained_models/' + file)
        hparams = checkpoint['model_hparams']
        model = WaveNet(2, feat_sizes=hparams['feat sizes'], depths=hparams['depths']).to(device)
        model.load_state_dict(checkpoint['model_state_dict'])

        predictions = evaluate_series(model, device, series, mode=mode)
        pred_list.append(predictions)
        model_num += 1
    
    return pred_list

def ensemble_voting(predictions, mode='hard'):
    """
    Takes a nested list of model predictions and computes voting results depending on
    mode selection.
    """

    # Stack predictions from each model
    stacked_preds = []
    for i in range(len(predictions)):
        stacked_preds.append(torch.stack(predictions[i], dim=0))

    # Stack predictions from all models
    stacked_preds = torch.stack(stacked_preds, dim=0).to('cpu') # (model num, num days, num steps) when mode = hard
                                                      # (model num, num classes, num days, num steps) when mode = soft

    if mode == 'hard':
        num_votes = torch.zeros((3, stacked_preds.shape[1], stacked_preds.shape[2]))

        # Awake votes
        indxs = torch.where(stacked_preds == 0)
        num_votes[0, indxs[1], indxs[2]] += 1
        # Asleep votes
        indxs = torch.where(stacked_preds == 1)
        num_votes[1, indxs[1], indxs[2]] += 1
        # Not wearing votes
        indxs = torch.where(stacked_preds == 2)
        num_votes[2, indxs[1], indxs[2]] += 1

        voted_predictions = torch.argmax(num_votes, dim=0) # (num_days, num steps)
    
    if mode == 'soft':
        summed_preds = torch.sum(stacked_preds, dim=0) # (num classes, num days, num steps)
        voted_predictions = torch.argmax(summed_preds, dim=0) # (num days, num steps)
    
    return voted_predictions

def get_series_predictions(series_df, index):
    """
    Returns the voted predictions of a series
    """
    id = series_df.series_id.unique()[index]

    series = torch.Tensor(np.reshape(series_df[series_df.series_id.isin([id])][['anglez', 'enmo']].to_numpy().T, (1, 2, -1)))

    predictions = series_predict(series, mode='hard')
    voted_preds = ensemble_voting(predictions, mode='hard')

    return voted_preds

def plot_series_preds(prediction, eval_dataset):
    onset_step, wakeup_step =  process_prediction(prediction)

    id = eval_dataset.series_id.unique()[index]

    series = eval_dataset[eval_dataset.series_id.isin([id])][['anglez', 'enmo']].to_numpy().T

    fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True, squeeze=True, figsize=(16, 8))
    steps = np.arange(series.shape[-1])
    # labels = eval_dataset[index][1].numpy()

    # accuracy = np.sum(np.array([prediction == labels]))/prediction.shape[0]
    # print('Example accuracy = {:.4f}'.format(accuracy))

    num_days = series.shape[1]//17280+1
    day_steps = []
    for day in range(num_days):
        day_steps.append(day*17280)

    ax1.plot(steps, series[0, :])
    ax1.fill_between(steps, y1=100, y2=0, where=prediction[0]==0, facecolor='green', alpha=.5)
    ax1.fill_between(steps, y1=100, y2=0, where=prediction[0]==1, facecolor='red', alpha=.5)
    ax1.fill_between(steps, y1=100, y2=0, where=prediction[0]==2, facecolor='blue', alpha=.5)
    # ax1.fill_between(steps, y1=0, y2=-100, where=labels==0, facecolor='lime', alpha=.5)
    # ax1.fill_between(steps, y1=0, y2=-100, where=labels==1, facecolor='orangered', alpha=.5)
    # ax1.fill_between(steps, y1=0, y2=-100, where=labels==2, facecolor='blue', alpha=.5)
    ax1.vlines(onset_step, -100, 100, colors='r', label='onset')
    ax1.vlines(wakeup_step, -100, 100, colors='b', label='wakeup')
    ax1.vlines(day_steps, -100, 100, colors='w', label='wakeup')
    ax1.set_ylabel('anglez freq')

    ax2.plot(steps, series[1, :])
    ax2.fill_between(steps, y1=2, y2=0, where=prediction[0]==0, facecolor='green', alpha=.5)
    ax2.fill_between(steps, y1=2, y2=0, where=prediction[0]==1, facecolor='red', alpha=.5)
    ax2.fill_between(steps, y1=2, y2=0, where=prediction[0]==2, facecolor='blue', alpha=.5)
    # ax2.fill_between(steps, y1=3, y2=2, where=labels==0, facecolor='lime', alpha=.5)
    # ax2.fill_between(steps, y1=3, y2=2, where=labels==1, facecolor='orangered', alpha=.5)
    # ax2.fill_between(steps, y1=3, y2=2, where=labels==2, facecolor='blue', alpha=.5)
    ax2.vlines(onset_step, 0, 3, colors='r')
    ax2.vlines(wakeup_step, 0, 3, colors='b')
    ax2.vlines(day_steps, 0, 3, colors='w')
    ax2.set_xlabel('step')
    ax2.set_ylabel('enmo')
    
    # day = 24
    # plt.xlim([17280*(day-1), 17280*(day+3)])

    plt.show()
    plt.close()

index = 0

In [ ]:
torch.cuda.empty_cache()
vote_preds = get_series_predictions(train_series, index)
plot_series_preds(vote_preds, train_series)
index += 1

# Post-processing (series evaluation)

In [ ]:
def series_processing(series_df):
    """
    Creates and processes the enseble predictions on an entire series. The predictions are chunked based on the noon to noon day and then the usual post-processing is performed on each chunk.
    """

    ids = series_df.series_id.unique()
    event_steps = []
    event_list = []
    row_ids = []
    series_ids = []
    
    series_num = 0
    for id in ids:
        print('Evaluating series ' + id)

        # Organize series data and chunk indices
        series = series_df[series_df.series_id.isin([id])]
        series_data = torch.Tensor(np.reshape(series[['anglez', 'enmo']].to_numpy().T, (1, 2, -1)))
        print(series.timestamp)
        noon_steps = series.step[series.timestamp.str.contains('12:00:00')].to_list()
        print(noon_steps)
        chunk_indxs = [series.step.iloc[0]] + [step for step in noon_steps] + [series.step.iloc[-1]]
        chunk_indxs = list(zip(chunk_indxs, chunk_indxs[1:]))
        
        # Account for case where first or last timestamp in series is at noon
        chunk_indxs = [chunk_indxs[i] for i in range(len(chunk_indxs))
                       if chunk_indxs[i][0] != chunk_indxs[i][1]]
        
        # Make predictions on series
        predictions = series_predict(series_data, mode='hard')
        voted_preds = ensemble_voting(predictions, mode='hard')
        # Iterate through noon indices and process chunks
        for i in range(len(chunk_indxs)):
            print(voted_preds[0][chunk_indxs[i][0]:chunk_indxs[i][1]].numpy().shape[0])
            pred_chunk = voted_preds[0][chunk_indxs[i][0]:chunk_indxs[i][1]].numpy()
            onset, wakeup = process_prediction(pred_chunk)
            if not np.isnan(onset):
                event_steps.append(onset)
                event_list.append('onset')
                event_steps.append(wakeup)
                event_list.append('wakeup')
                series_ids.append(id)
                series_ids.append(id)

        series_num += 1
        if series_num > 10:
            break

    row_ids = [int(i) for i in range(len(event_list))]
    scores = [1.0 for _ in range(len(event_list))]

    # Create submission dataframe
    submission_df = pd.DataFrame({'row_id': row_ids,
                                  'series_id': series_ids, 
                                  'step': event_steps, 
                                  'event': event_list, 
                                  'score': scores})

    return submission_df

In [ ]:
submission_df = series_processing(train_series)

In [ ]:
test = torch.Tensor([0,1,3,6,1,1])
test2 = torch.Tensor([1,1,3,4,8,1])
torch.where((test == 1) & (test2 == 1))[0].shape[0]/torch.where(test2 == 1)[0].shape[0]

In [ ]:
from math import sqrt, pi, exp
SIGMA = 90
def gauss(mu, sigma=SIGMA):
    # guassian distribution function
    r = [mu - 360, mu - 300, mu - 240, mu - 180, mu - 150, mu - 120, mu - 90, mu - 60, mu - 36, mu - 12, 
         mu, mu + 12, mu + 36, mu + 60, mu + 90, mu + 120, mu + 150, mu + 180, mu + 240, mu + 300,  mu + 360]
    return [1 / (sigma * sqrt(2*pi)) * exp(-float(x - mu)**2/(2*sigma**2)) for x in r], r